In [1]:
import requests  # Biblioteca padrão para fazer chamadas HTTP (GET) na web
import json      # Biblioteca para manipular o formato de texto JSON
from hdfs import InsecureClient # Cliente para conectar no Hadoop

# 1. Configuração do Cliente HDFS
client = InsecureClient('http://namenode:9870', user='root')

# Função para cada url
def ingestao_bronze(url_api, nome_arquivo_destino):
    print(f"Baixando dados de: {nome_arquivo_destino}...")
    
    # 2. Extração (Extract)
    # requests.get(url): Vai na internet e bate na porta da API.
    response = requests.get(url_api)
    
    # .json(): Pega o texto que veio da internet e transforma em Lista/Dicionário Python.
    # ['value']: A API da OMS coloca os dados dentro de uma chave chamada 'value', só queremos isso.
    dados = response.json()['value']
    
    # 3. Carga no HDFS (Load)
    # caminho_completo: Onde o arquivo vai morar dentro do Cluster.
    caminho_completo = f"/datalake/bronze/{nome_arquivo_destino}.json"
    
    # client.write: Abre um canal de escrita direto no HDFS.
    # encoding='utf-8': Garante que acentos não quebrem.
    # overwrite=True: Se rodar de novo, apaga o velho e põe o novo.
    with client.write(caminho_completo, encoding='utf-8', overwrite=True) as writer:
        # json.dump: Pega os dados da memória RAM e escreve no arquivo aberto.
        json.dump(dados, writer)
        
    print(f"✅ Arquivo salvo no HDFS: {caminho_completo}")

# Executando a função
URL_SUICIDIO = "https://ghoapi.azureedge.net/api/SDGSUICIDE?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY' and Dim2 eq 'AGEGROUP_YEARSALL'"
URL_DEPRESSAO = "https://ghoapi.azureedge.net/api/MH_12?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY'"

ingestao_bronze(URL_SUICIDIO, "raw_suicidio")
ingestao_bronze(URL_DEPRESSAO, "raw_depressao")


Baixando dados de: raw_suicidio...
✅ Arquivo salvo no HDFS: /datalake/bronze/raw_suicidio.json
Baixando dados de: raw_depressao...
✅ Arquivo salvo no HDFS: /datalake/bronze/raw_depressao.json
